# Disease Tagging Tutorial

In this example, we'll be writing an application to extract *mentions of* diseases from Pubmed abstracts, using annotations from the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial, which has 5 parts, walks through the process of constructing a model to classify _candidate_ disease mentions as either true (i.e., that it is truly a mention of a disease) or false.

## Part V: Evaluating the Model on the Test Set

In this final part of the tutorial, we will reload the model developed in Part IV and test it on the test `CandidateSet`.

A benchmark score for this task, using [TaggerOne](http://www.ncbi.nlm.nih.gov/pubmed/27283952), is **79.6 F1**; we'll see that we will come within 1-2 points F1 score of this already!  (And, if we utilize additional unlabeled data--not covered in this tutorial, but easy to implement!--we can match or exceed it!)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Disease` `Candidate` subclass from Parts II, III, and IV.

In [ ]:
from snorkel.models import candidate_subclass

Disease = candidate_subclass('Disease', ['disease'])

## Loading Test `CandidateSet`

We reload the test `CandidateSet` object from the previous parts of the tutorial.

In [ ]:
from snorkel.models import CandidateSet

test = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Test Candidates').one()

## Automatically Creating Features
We also generate features for the test set.

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

Note that we use the training features feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

In [ ]:
%time F_test = feature_manager.update(session, test, 'Train Features', False)

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_test = feature_manager.load(session, test, 'Train Features')

## Reloading the Discriminative Model

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
%time disc_model.load(session, "Discriminative Params")

## Evaluating on the Test `CandidateSet`

First, we load the test set labels and gold candidates we made in Part III.

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()
L_test = label_manager.load(session, test, "CDR Test Labels -- Gold")

In [ ]:
gold_test_set = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Test Candidates -- Gold').one()

In [ ]:
tp, fp, tn, fn = disc_model.score(F_test, L_test, gold_test_set)

You've completed the tutorial!